In [17]:
!pip install transformers pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd


df_posts = pd.read_csv('./drive/MyDrive/fb_posts_emotion.csv')
df_posts.head()

,Unnamed: 0,created_time,description,link,message,page_id,post_id,react_angry,react_haha,react_like,react_love,react_sad,react_wow,scrape_time,shares,post_id_split,n_anger,n_comments
0,0,2017-07-14T14:30:59+0000,NaN,https://www.facebook.com/bbcnews/videos/101548...,We are #LIVE outside the National Rifle Associ...,228735667216,228735667216_10154890879532217,54,24,993,144,12,24,2017-07-14 11:01:24.379857,139,10154890879532217,2,100
1,1,2017-07-14T14:20:59+0000,,http://bbc.in/2talMsx,UPDATE: \r\n-2 Ukrainian tourists killed in st...,228735667216,228735667216_10154890968202217,172,8,994,11,783,264,2017-07-14 11:01:24.379857,680,10154890968202217,2,99
2,2,2017-07-14T13:40:38+0000,NaN,https://www.facebook.com/bbcnews/videos/101548...,Proms: Come with us on a tour of the Royal Alb...,228735667216,228735667216_10154890852247217,5,12,2034,369,6,45,2017-07-14 11:01:24.379857,395,10154890852247217,0,95
3,3,2017-07-14T12:55:45+0000,NaN,https://www.facebook.com/bbcnews/videos/142678...,Thousands say their final goodbyes to Bradley ...,228735667216,228735667216_1426789250735491,6,0,2262,754,1989,11,2017-07-14 11:01:24.379857,542,1426789250735491,1,95
4,4,2017-07-14T12:45:00+0000,NaN,https://www.facebook.com/bbcnews/videos/101548...,"Despite safety warnings, this beach near an ai...",228735667216,228735667216_10154890645702217,65,513,4336,54,128,815,2017-07-14 11:01:24.379857,1956,10154890645702217,9,100


In [19]:
def anger_pct(x):
    total_reacts = x.react_angry + x.react_haha + x.react_sad + x.react_wow + x.react_love + x.react_like

    if total_reacts == 0:
        return 0

    return x.react_angry / total_reacts

def anger_comment_pct(x):
    total_reacts = x.react_angry + x.react_haha + x.react_sad + x.react_wow + x.react_love + x.react_like + x.n_comments

    if total_reacts == 0:
        return 0

    return (x.react_angry + x.n_anger) / total_reacts

def anger_class(x):
    if x.anger_pct <= 1e-2:
       return 0

    if 1e-2 < x.anger_pct <= 1e-1:
       return 1

    return 2

# df_posts['anger_pct'] = df_posts.apply(anger_pct, axis=1)
df_posts['anger_pct'] = df_posts.apply(anger_comment_pct, axis=1)
df_posts['anger_class'] = df_posts.apply(anger_class, axis=1)
df_posts.anger_class.value_counts()


0    8056
1    5781
2    5284
Name: anger_class, dtype: int64

In [20]:
import numpy as np
np.random.seed(1234)

df_not_angry = df_posts[df_posts.anger_class == 0]
df_angry_middle = df_posts[df_posts.anger_class == 1]
df_angry_max = df_posts[df_posts.anger_class == 2]

df_not_angry = df_not_angry.sample((len(df_angry_middle) + len(df_angry_max)) // 2)
df_data = df_not_angry.append(df_angry_middle).append(df_angry_max)

df_data.anger_class.value_counts()

1    5781
0    5532
2    5284
Name: anger_class, dtype: int64

In [21]:
indices = list(range(len(df_data)))
np.random.shuffle(indices)

print(indices)

idx_train = indices[:int(.8 * len(indices))]
idx_val = indices[int(.8 * len(indices)):int(.9 * len(indices))]
idx_test = indices[int(.9 * len(indices)):]

print(len(idx_train))
print(len(idx_val))
print(len(idx_test))

[6874, 16179, 7736, 3008, 10022, 2009, 2971, 11025, 6169, 14206, 9493, 7010, 9414, 14172, 2479, 12342, 14078, 12468, 13, 7938, 14550, 8497, 8132, 14368, 4221, 9271, 7065, 1624, 3263, 4534, 4317, 13164, 11875, 4204, 6348, 5481, 16028, 386, 4133, 12486, 13138, 5618, 14862, 9437, 7161, 2257, 15076, 9235, 16207, 5135, 2334, 5647, 13221, 5780, 11045, 6159, 3550, 16030, 7891, 8715, 1271, 15780, 11878, 4266, 8821, 10193, 11050, 4491, 7795, 9569, 8780, 4679, 15020, 7812, 12080, 5228, 3804, 8024, 13354, 2752, 3297, 6750, 15574, 12217, 12921, 11742, 9186, 14028, 9351, 1327, 11932, 10367, 1823, 14753, 8861, 11435, 15472, 8285, 7194, 6375, 1710, 16273, 5380, 2223, 5699, 6741, 3188, 16315, 2455, 1247, 16082, 13775, 16069, 1374, 6956, 14406, 15572, 7346, 8250, 8018, 7614, 6709, 11771, 6228, 4879, 16464, 6133, 12564, 236, 15722, 13866, 10370, 1218, 15746, 11696, 10531, 7284, 12934, 218, 13988, 15395, 1213, 6848, 11424, 12268, 7433, 7124, 5165, 2596, 15608, 3347, 8123, 15967, 15174, 8148, 9364, 13321,

In [22]:
df_train = df_data.iloc[idx_train]
df_val = df_data.iloc[idx_val]
df_test = df_data.iloc[idx_test]

print(len(df_train), len(df_val), len(df_test))

13277 1660 1660


In [23]:
import torch

class FBNewsDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        super().__init__()
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        return row.message, row.anger_class


ds = FBNewsDataset(df_train)
ds[0]


('Beyoncé has finally released the first photo of her twins and confirmed their rumored names.',
 1)

In [ ]:
from transformers import AutoTokenizer

class TokenizerCollateFn:
    def __init__(self, tokenizer_name, max_length=400):
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        self.max_length = max_length

    def _encode(self, sentence):
        return self.tokenizer(
            sentence,
            max_length=self.max_length,
            padding='longest',
            return_tensors='pt',
            truncation=True
        )

    def __call__(self, batch):
        batch_x = []
        batch_y = []

        for x, y in batch:
            batch_x.append(x)
            batch_y.append(y)

        t_out = self._encode(batch_x)

        return t_out.input_ids, t_out.attention_mask, torch.LongTensor(batch_y)

t = TokenizerCollateFn('bert-base-cased')
t([ds[0], ds[1], ds[2]])

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

(tensor([[  101,  1249,  1103,  2733, 10083,  5965,  1207, 16291,   117,  2757,
           1110, 27066,  1293,  1277,  3748,  1106,  1660,  1697,  8499,  1106,
           9046,  7166, 19344,  1222,  1103,  6716,   119,   102,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
         [  101,  1247,  1132,  1374,  1167,  7271, 12644,  1190,  1103,  1849,
           1107,  1293,  2170,   118,  4038,  1208,  1440,  1120,  2560,  5582,
            119,  3561, 11811,  4253,  1193,   117,  1122,   787,   188,  1562,
           1112,   170,  7950,  1268,   119,   102,     0,     0,     0,     0],
         [  101,  1332,  1528,  8861, 10516,  2508,  4661,  1883,  3891,  1103,
           2061,  1585,  1171,  1107,  1345,   117,  1131,  1105,  1697,  5554,
           8499,  1238,   112,   189,  5854,  1493,   119,  1262,  1122,  1108,
          28117,  1358,  1358,  1358,  1358, 26939,  1197,  9540,   119,   102]]),
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
from transformers import BertPreTrainedModel, BertModel
from torch import nn

class BertForClassificationError(Exception):
    pass

class BertForClassification(BertPreTrainedModel):
    def __init__(self, config, n_labels=3):
        super().__init__(config)
        
        if n_labels < 3:
            raise BertForClassificationError(f'n_labels must be at least 3')

        self.bert = BertModel(config)
        self.dropout = nn.Dropout(p=config.attention_probs_dropout_prob)
        self.dense = nn.Linear(
            in_features=self.bert.pooler.dense.out_features, 
            out_features=n_labels
        )

        self.loss_fct = nn.CrossEntropyLoss()
    
    def forward(self, input_ids, attention_mask, *args, **kwargs):
        outs = self.bert(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            *args, 
            **kwargs
        )

        dropped = self.dropout(outs[1])
        logits = self.dense(dropped)

        return logits, outs 



b = BertForClassification.from_pretrained('bert-base-cased')
dl = torch.utils.data.DataLoader(ds, collate_fn=t, shuffle=True, num_workers=2, batch_size=1)
input_ids, attention_mask, labels = next(iter(dl))
out = b(input_ids=input_ids, attention_mask=attention_mask)
out

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForClassification were not initialized from the model checkpoint at bert-base-cased and are newly ini

(tensor([[-0.0559, -0.0872,  0.1208]], grad_fn=<AddmmBackward0>),
 BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.2297,  0.0916, -0.1735,  ..., -0.4630,  0.4994,  0.3716],
          [ 0.5016, -0.3286, -0.2743,  ..., -0.3369,  0.3169,  0.6219],
          [-0.5587, -0.5299, -0.2659,  ..., -0.4136, -0.3704,  0.3004],
          ...,
          [-0.7414, -0.8435,  0.1400,  ...,  0.5365, -0.3181,  1.9749],
          [ 0.4540, -0.2033, -0.4668,  ..., -0.0697,  0.3810,  0.9019],
          [ 0.1295,  0.2128, -0.5489,  ..., -0.5868,  0.6282, -0.1199]]],
        grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.8155,  0.5565,  1.0000, -0.9958,  0.9717,  0.8061,  0.9892, -0.9738,
          -0.9885, -0.6768,  0.9939,  0.9992, -0.9935, -1.0000,  0.6549, -0.9840,
           0.9937, -0.6528, -1.0000, -0.1547, -0.2481, -1.0000,  0.3273,  0.9104,
           0.9868,  0.0792,  0.9947,  1.0000,  0.8750,  0.1413,  0.3428, -0.9949,
           0.6774, -0.9997,  0.2673,

In [ ]:
import pytorch_lightning as pl

from torch.nn import functional as F
from torch.utils.data import DataLoader

from sklearn.metrics import accuracy_score

class FBNewsModel(pl.LightningModule):
    def __init__(self):
        super().__init__()

        self.model = BertForClassification.from_pretrained('bert-base-cased')
        
    def forward(self, *args, **kwargs):
        return self.model(*args, **kwargs)

    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch
        logits, _ = self.model(input_ids=input_ids, attention_mask=attention_mask)
        loss = F.cross_entropy(logits, labels)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch
        logits, _ = self.model(input_ids=input_ids, attention_mask=attention_mask)
        
        acc = accuracy_score(logits.argmax(dim=-1).cpu(), labels.cpu())
        self.log('val_batch_acc', acc, prog_bar=True)

        return acc

    def test_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch
        logits, _ = self.model(input_ids=input_ids, attention_mask=attention_mask)
        
        acc = accuracy_score(logits.argmax(dim=-1).cpu(), labels.cpu())
        self.log('test_batch_acc', acc, prog_bar=True)

        return acc

    def validation_epoch_end(self, outputs):
        self.log('mean_val_acc', np.mean(outputs), prog_bar=True)

    def test_epoch_end(self, outputs):
        self.log('mean_test_acc', np.mean(outputs), prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)


def test_model():
    dl_overfit = DataLoader(dataset=FBNewsDataset(df_train), batch_size=10, shuffle=False, collate_fn=t)
    trainer = pl.Trainer(
        gpus=1,
        overfit_batches=1,
        max_epochs=50
    )
    model = FBNewsModel()
    trainer.fit(model, train_dataloaders=dl_overfit, val_dataloaders=dl_overfit)

# test_model()

In [24]:
torch.manual_seed(1234)

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    save_top_k=3,
    monitor="mean_val_acc",
    mode="max",
    dirpath="./drive/MyDrive/checkpoints",
    filename="bert-class-comment-{epoch:02d}-{mean_val_acc:0.3f}",
)

trainer = pl.Trainer(
    gpus=1,
    max_epochs=30,
    callbacks=[checkpoint_callback]
)

dl_train = DataLoader(FBNewsDataset(df_train), shuffle=True, batch_size=20, collate_fn=t, num_workers=2)
dl_val = DataLoader(FBNewsDataset(df_val), shuffle=False, batch_size=20, collate_fn=t, num_workers=2)
dl_test = DataLoader(FBNewsDataset(df_test), shuffle=False, batch_size=20, collate_fn=t, num_workers=2)

# model = FBNewsModel()
# trainer.fit(model, train_dataloaders=dl_train, val_dataloaders=dl_val)
model = FBNewsModel.load_from_checkpoint("./drive/MyDrive/checkpoints/bert-class-comment-epoch=13-mean_val_acc=0.341.ckpt")
trainer.test(model, dl_test)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:446: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predic

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      mean_test_acc         0.3463855421686747
     test_batch_acc         0.3463855421686747
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_batch_acc': 0.3463855421686747, 'mean_test_acc': 0.3463855421686747}]